<a href="https://colab.research.google.com/github/calopezj/Accidentalidad2020/blob/master/2020_Accidentalidad_An%C3%A1lisis_espacial_(ejecutado).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import datetime
import folium
from folium.map import *
from folium import plugins
from folium.plugins import MeasureControl
from folium.plugins import FloatImage

In [3]:
pdAccidentalidad = pd.read_csv(
    "https://raw.githubusercontent.com/calopezj/Accidentalidad/master/accidentalidad.csv",
    sep = ';',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.')     # separador de los decimales para números
pdAccidentalidad.head()

,@timestamp,CBML,EXPEDIENTE,tipo,Direccion encasillada,CLASE_ACCIDENTE,location,Mes,Año,comuna,tags,DIRECCION,host,path,Numcomuna,Diseño,barrio,GRAVEDAD_ACCIDENTE,FECHA_ACCIDENTE,y,x,@version,Id,NRO_RADICADO
0,2019-06-04T20:44:03.039Z,708.0,A1484623,Malla vial,CR 080 079 C 000 00000,Choque,"[-75.5846584729, 6.28257144516]",2,2014,Robledo,[u'incidentes'],CR 80 CL 79 C,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,07,Tramo de via,Altamira,Solo daños,2014-02-14T22:30:00.000Z,1186726.32,833206.79,1,5251,1429648
1,2019-06-04T20:44:03.040Z,502.0,A1484637,Malla vial,CR 064 C 110 000 00000,Choque,"[-75.5617778582, 6.3034471604]",2,2014,Castilla,[u'incidentes'],CR 64 C CL 110,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,05,Tramo de via,Las Brisas,Solo daños,2014-02-14T13:40:00.000Z,1189028.44,835746.06,1,5252,1429659
2,2019-06-04T20:44:03.040Z,1510.0,A1485063,Malla vial,CR 050 S 008 000 00000,Choque,"[-75.5801585514, 6.20022627101]",2,2014,Guayabal,[u'incidentes'],CR 50 CL 8 Sur,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,15,Tramo de via,Guayabal,Solo daños,2014-02-14T19:15:00.000Z,1177615.47,833678.92,1,5253,1429753
3,2019-06-04T20:44:03.041Z,1603.0,A1485168,Malla vial,CR 080 030 000 00000,Choque,"[-75.6006896809, 6.23107041857]",2,2014,Belén,[u'incidentes'],CR 80 CL 30,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,16,Tramo de via,Belén,Solo daños,2014-02-15T02:45:00.000Z,1181034.11,831415.73,1,5256,1429815
4,2019-06-04T20:44:03.042Z,412.0,A1484257,Malla vial,CL 090 048 000 00000,Atropello,"[-75.5557424542, 6.28209069323]",2,2014,Aranjuez,[u'incidentes'],CL 90 CR 48,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,04,Tramo de via,La Piñuela,Con heridos,2014-02-15T05:15:00.000Z,1186664.01,836407.42,1,5259,1429191


##Datos básicos

In [4]:
print(len(pdAccidentalidad))
pdAccidentalidad.columns

56914


Index(['@timestamp', 'CBML', 'EXPEDIENTE', 'tipo', 'Direccion encasillada',
       'CLASE_ACCIDENTE', 'location', 'Mes', 'Año', 'comuna', 'tags',
       'DIRECCION', 'host', 'path', 'Numcomuna', 'Diseño', 'barrio',
       'GRAVEDAD_ACCIDENTE', 'FECHA_ACCIDENTE', 'y', 'x', '@version', 'Id',
       'NRO_RADICADO'],
      dtype='object')

## Subset con los datos necesarios:
1. Cree un subset con solo los datos correpondientes a las columnas 'location', 'FECHA_ACCIDENTE', 'Año', 'CLASE_ACCIDENTE', 'barrio'

In [5]:
pdAccidentalidad_new_SubSet = pdAccidentalidad [['location', 'FECHA_ACCIDENTE', 'Año', 'CLASE_ACCIDENTE', 'barrio']]
pdAccidentalidad_new_SubSet.head()

,location,FECHA_ACCIDENTE,Año,CLASE_ACCIDENTE,barrio
0,"[-75.5846584729, 6.28257144516]",2014-02-14T22:30:00.000Z,2014,Choque,Altamira
1,"[-75.5617778582, 6.3034471604]",2014-02-14T13:40:00.000Z,2014,Choque,Las Brisas
2,"[-75.5801585514, 6.20022627101]",2014-02-14T19:15:00.000Z,2014,Choque,Guayabal
3,"[-75.6006896809, 6.23107041857]",2014-02-15T02:45:00.000Z,2014,Choque,Belén
4,"[-75.5557424542, 6.28209069323]",2014-02-15T05:15:00.000Z,2014,Atropello,La Piñuela


## Identificar barrios con mayor cantidad de accidentes
2. Liste los 10 barrios en los que mas accidentes se han presentado

In [9]:
pdAccidentalidad_new_SubSet.groupby('barrio').size().sort_values(ascending = False).head(10)

barrio
La Candelaria                    1238
Campo Amor                       1091
Perpetuo Socorro                 1055
Caribe                           1042
Los Conquistadores                908
Barrio Colón                      905
Santa Fé                          821
Cabecera San Antonio de Prado     815
Villa Nueva                       787
Carlos E. Restrepo                724
dtype: int64

## Accidentes de un barrio determinado
3. Construya un subset con los accidentes ocurridos en un barrio determinado.

In [12]:
pdAccidentes = pdAccidentalidad_new_SubSet[pdAccidentalidad_new_SubSet['barrio'] == 'Los Conquistadores']
len(pdAccidentes)

908

### Extraemos la posición de cada accidente
4. A partir de la columna "location", cree dos columnas nuevas, una con la longitud y otra con la latitud correspondiente a las coordenadas del accidente.

In [14]:
lista = pdAccidentes["location"].str.split(",", n = 1, expand = True)
pdAccidentes["longitud"] = lista[0]
pdAccidentes["latitud"] = lista[1]
#pdAccidentes.loc[:, "longitud3"] = lista[0]

pdAccidentes


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,location,FECHA_ACCIDENTE,Año,CLASE_ACCIDENTE,barrio,longitud,latitud
31,"[-75.5778924993, 6.23961416776]",2014-02-15T20:20:00.000Z,2014,Choque,Los Conquistadores,[-75.5778924993,6.23961416776]
47,"[-75.589313202, 6.2397866157]",2014-02-14T10:55:00.000Z,2014,Choque,Los Conquistadores,[-75.589313202,6.2397866157]
85,"[-75.5790498927, 6.24191559797]",2014-02-18T17:00:00.000Z,2014,Choque,Los Conquistadores,[-75.5790498927,6.24191559797]
121,"[-75.5854551105, 6.24105317866]",2014-02-18T20:00:00.000Z,2014,Choque,Los Conquistadores,[-75.5854551105,6.24105317866]
143,"[-75.587516188, 6.23927100344]",2014-02-05T14:30:00.000Z,2014,Otro,Los Conquistadores,[-75.587516188,6.23927100344]
...,...,...,...,...,...,...,...
56513,"[-75.5786297845, 6.24195837799]",2019-02-26T11:30:00.000Z,2019,Choque,Los Conquistadores,[-75.5786297845,6.24195837799]
56529,"[-75.5831360679, 6.23940141843]",2019-02-26T17:00:00.000Z,2019,Choque,Los Conquistadores,[-75.5831360679,6.23940141843]
56710,"[-75.5785464683, 6.23998075728]",2019-03-26T11:30:00.000Z,2019,Caida Ocupante,Los Conquistadores,[-75.5785464683,6.23998075728]
56856,"[-75.5875625807, 6.23922612473]",2019-03-30T01:00:00.000Z,2019,Caida Ocupante,Los Conquistadores,[-75.5875625807,6.23922612473]


### Limpiamos caracteres innecesarios

In [15]:
pdAccidentes["longitud"] = pdAccidentes["longitud"].str.replace("[","")
pdAccidentes["latitud"] = pdAccidentes["latitud"].str.replace("]","")

pdAccidentes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,location,FECHA_ACCIDENTE,Año,CLASE_ACCIDENTE,barrio,longitud,latitud
31,"[-75.5778924993, 6.23961416776]",2014-02-15T20:20:00.000Z,2014,Choque,Los Conquistadores,-75.5778924993,6.23961416776
47,"[-75.589313202, 6.2397866157]",2014-02-14T10:55:00.000Z,2014,Choque,Los Conquistadores,-75.589313202,6.2397866157
85,"[-75.5790498927, 6.24191559797]",2014-02-18T17:00:00.000Z,2014,Choque,Los Conquistadores,-75.5790498927,6.24191559797
121,"[-75.5854551105, 6.24105317866]",2014-02-18T20:00:00.000Z,2014,Choque,Los Conquistadores,-75.5854551105,6.24105317866
143,"[-75.587516188, 6.23927100344]",2014-02-05T14:30:00.000Z,2014,Otro,Los Conquistadores,-75.587516188,6.23927100344
...,...,...,...,...,...,...,...
56513,"[-75.5786297845, 6.24195837799]",2019-02-26T11:30:00.000Z,2019,Choque,Los Conquistadores,-75.5786297845,6.24195837799
56529,"[-75.5831360679, 6.23940141843]",2019-02-26T17:00:00.000Z,2019,Choque,Los Conquistadores,-75.5831360679,6.23940141843
56710,"[-75.5785464683, 6.23998075728]",2019-03-26T11:30:00.000Z,2019,Caida Ocupante,Los Conquistadores,-75.5785464683,6.23998075728
56856,"[-75.5875625807, 6.23922612473]",2019-03-30T01:00:00.000Z,2019,Caida Ocupante,Los Conquistadores,-75.5875625807,6.23922612473


## Conversion de tipos de datos
Dado que la herramienta de georreferenciación recibe como parámetros coordenadas numericas, cambie el formato de string a float de los datos latitud y longitud de cada registro.

In [17]:
pdAccidentes["longitud"] = pdAccidentes["longitud"].astype(float)
pdAccidentes["latitud"] = pdAccidentes["latitud"].astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Promedio de coordenadas
La herramienta de georreferenciación requiere un punto central para dibujar el mapa base. Dicho punto puede ser el promedio de latitudes y longitudes de los diferentes accidentes.

In [18]:
LongitudProm = pdAccidentes["longitud"].mean()
print(LongitudProm)
LatitudProm = pdAccidentes["latitud"].mean()
print(LatitudProm)


-75.58303720071171
6.240855986593889


# Analisis Espacial

In [21]:
m = folium.Map(
    location=[float(LatitudProm), float(LongitudProm)],
    #location=[6.24085598659, -75.58303720071],
    #tiles='Stamen Toner',
    tiles='Stamen Terrain',
    zoom_start=16
)
m



In [22]:
tooltip = 'Accidente'

for index, row in pdAccidentes.iterrows():

  folium.Marker([float(row['latitud']), float(row['longitud'])], popup='Accidente', tooltip=tooltip).add_to(m)

m